# 해시법 (오픈 주소법)

오픈 주소법은 충돌이 발생할 때 해결하는 방법으로 재해시를 수행하여 빈 버킷을 찾는 방법이며, 닫힌 해시법이라고도 한다.

오픈 주소법은 빈 버킷이 나올 때까지 재해시를 반복하므로 선형탐사법이라고도 한다.

### 원소 추가하기

재해시 함수는 자유롭게 정할 수 있다. 예를 들어 키에 1을 더하여 전체 개수로 나눈 나머지를 사용하는 등의 방법이 있다.

### 원소 삭제하기

단순히 인덱스에 해당하는 버킷만을 삭제하면 안된다. 해당 인덱스에서 재해시 되었을 수 있기 때문에 단순히 삭제만 한다면 그 인덱스로부터 재해시된 데이터가 있는지 알 수 없다. 따라서 삭제가 되었음을 표시해 둠으로써 원소를 검색할 때 제대로 검색을 완료할 수 있다.

(ex)

![오픈 주소법](https://user-images.githubusercontent.com/40788586/90313618-e3c1f500-df48-11ea-9528-17bd5d3c9fad.png)

위의 그림과 같이 18을 13으로 나눈 나머지로 해시하면 5가 된다. 이때 value로 5를 갖는 원소가 이미 key 5번에 들어가 있기 때문에 충돌이 발생한다. 따라서 (18+1) % 13으로 해시하여 인덱스 6번을 확인하고, 여기서도 충돌이 발생하여 7번 버킷에 삽입된다. 이렇게 되면 5를 삭제해버리고 18을 검색할 경우 18을 해시한 결과인 5의 원소가 없으므로 18이 버킷에 들어있지 않다고 생각할 수 있다. 따라서 삭제가 되었음을 표시해줌으로써 18을 찾기 위해 재해시를 하도록 유도할 수 있다.

## 오픈 주소법으로 해시 함수 구현

In [3]:
# 모듈 import

from __future__ import annotations
from typing import Any, Type
from enum import Enum
import hashlib

In [4]:
# 버킷의 속성
class Status(Enum):
    OCCUPIED = 0 # 데이터 저장
    EMPTY = 1 # 비어 있음
    DELETED = 2 # 삭제 완료

In [5]:
# 해시를 구성하는 버킷
class Bucket:
    def __init__(self, key: Any = None, value: Any = None,
                        stat: Status = Status.EMPTY)->None:
        self.key = key
        self.value = value
        self.stat = stat
    
    def set(self, key: Any, value: Any, stat: Status)->None:
        self.key = key
        self.value = value
        self.stat = stat
    
    def set_status(self, stat: Status)->None:
        self.stat = stat
    
# 오픈 주소법으로 구현하는 해시 클래스
class OpenHash:
    def __init__(self, capacity: int) -> None:
        self.capacity = capacity
        self.table = [None] * self.capacity
        
    def hash_value(self, key: Any) -> int:
        """해시값을 구함"""
        if isinstance(key, int):
            return key % self.capacity
        return (int(hashlib.md5(str(key).encode().hexdigest(), 16) % self.capacity))
    
    def rehash_value(self, key: Any) -> int:
        """재해시값을 구함"""
        return(self.hash_value(key) + 1) % self.capacity
    
    def search_node(self, key: Any) -> Any:
        hash = self.hash_value(key)
        p = self.table[hash]
        
        for i in range(self.capacity):
            if p.stat == Status.EMPTY:
                break
            elif p.stat == Status.OCCUPIED and p.key == key:
                return p
            hash = self.rehash_value(hash)
            p = self.table[hash]
        return None
    
    def search(self, key: Any)->Any:
        p = self.search_node(key)
        if p is not None:
            return p.value
        else:
            return None
    
    def add(self, key: Any, value: Any)->bool:
        if self.search(key) is not None:
            return False
        
        hash = self.table[hash]
        for i in range(self.capacity):
            if p.stat == Status.EMPTY or p.stat == Status.DELETED:
                self.table[hash] = Bucket(key, value, Stuatus.OCCUPIED)
                return True
            hash = self.rehash_value(hash)
            p = self.table[hash]
        return false
    
    def remove(self, key: Any)->bool:
        p = self.search_node(key)
        if p is None:
            return False
        p.set_status(Status.DELETED)
        return True